# 1. Combining resized Frames in Video (if need)

In [ ]:
import cv2
import os
import glob
# Setting pathway
image_folder = '/Users/yz/Files/CODES/INTERN/Bottle-1/test/images'  # Pathway of images
video_path = '/Users/yz/Files/CODES/INTERN/Bottle-1/test/output_video.mp4'  # Pathway of output

# Setting attributes
images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape
fps = 30  # fps of output

# Create VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
video = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

# Adding frames in Video
for image in sorted(images):
    video.write(cv2.imread(os.path.join(image_folder, image)))

# Release storage
cv2.destroyAllWindows()
video.release()

print("Video created successfully!")


# 2.Require YOLOv8 Model via API

In [ ]:
!pip install roboflow

In [ ]:
import cv2
from roboflow import Roboflow

# Assuming you've already installed Roboflow with pip install roboflow
api_key = "zt26bVuyi5xf9yizf4i2"
workspace = "12-jqvcq"
project_name = "bottle-oebjd"
version_number = 1


# Initialize Roboflow model
rf = Roboflow(api_key=api_key)
project = rf.workspace(workspace).project(project_name)
version = project.version(version_number)
model = version.model


# Open the video
cap = cv2.VideoCapture('/Users/yz/Files/CODES/INTERN/Bottle-1/test/AA.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/Users/yz/Files/CODES/INTERN/Bottle-1/test/outAA.mp4', fourcc, fps, (width, height))


# 3.1 Objecting and Tracking items start with 0 counts without Interface

In [ ]:
import cv2
from roboflow import Roboflow
import numpy as np

# Initialize your video capture, model, and other necessary parts above this code

mid_line = height*9// 15
line_color = (255, 0, 0)  # Red color in BGR
line_thickness = 2

# Dictionary to track objects
trackers = {}


item_appear_under_line = int(0)
item_appear_over_line = int(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    cv2.line(frame, (0, mid_line), (width, mid_line), line_color, line_thickness)

    # Apply model to each frame
    predictions = model.predict(frame)
    for prediction in predictions:
        x, y, w, h = int(prediction['x']), int(prediction['y']), int(prediction['width']), int(prediction['height'])
        center_y = y + h
        label = prediction['class']
        id_found = None

        # Find the closest existing tracker (simple overlap or centroid distance)
        for obj_id, (obj_label, obj_bbox, obj_last_y, obj_crossed) in trackers.items():
            if label == obj_label and not obj_crossed and np.abs(obj_last_y - center_y) < 300:  # 200 pixels threshold
                id_found = obj_id
                break

        if id_found is None:
            # New object detected
            new_id = max(trackers.keys(), default=-1) + 1
            trackers[new_id] = (label, (x, y, w, h), center_y, False)
        else:
            # Update existing tracker
            _, _, obj_last_y, obj_crossed = trackers[id_found]
            if center_y < mid_line and obj_last_y >= mid_line and not obj_crossed:
                item_count[label] += 1  # Update this logic based on your needs
                trackers[id_found] = (label, (x, y, w, h), center_y, True)
            elif center_y > mid_line and obj_last_y <= mid_line and not obj_crossed:
                item_count[label] -= 1
                trackers[id_found] = (label, (x, y, w, h), center_y, True)
            else:
                trackers[id_found] = (label, (x, y, w, h), center_y, obj_crossed)

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} {prediction['confidence']:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display item counts in the bottom-left corner of the frame
    text_position = (10, height - 10)
    for key, value in item_count.items():
        display_text = f"{key}: {value}"
        cv2.putText(frame, display_text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        text_position = (text_position[0], text_position[1] - 30)  # Adjust this value based on your font size

    # Write the frame into the file
    out.write(frame)

    # Clean up trackers that are no longer in view
    trackers = {obj_id: v for obj_id, v in trackers.items() if v[2] > 0 or v[2] < height}

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()
print("Item counts:", item_count)


# 3.2 Objecting and Tracking items with self-identified Interface

In [ ]:
import cv2
from roboflow import Roboflow
import numpy as np
import tkinter as tk
from tkinter import ttk

# Predefined list of items (you can modify this list or read from a file)
KNOWN_ITEMS = ['Bottle', 'R_Lunch']

class ItemCountInitializer(tk.Toplevel):
    def __init__(self, parent):
        super().__init__(parent)
        self.title("Initialize Item Counts")
        self.geometry("300x400")
        self.item_counts = {}

        self.create_widgets()

    def create_widgets(self):
        for item in KNOWN_ITEMS:
            frame = ttk.Frame(self)
            frame.pack(pady=5, padx=10, fill='x')

            label = ttk.Label(frame, text=item)
            label.pack(side='left')

            count_var = tk.IntVar(value=0)
            spinbox = ttk.Spinbox(frame, from_=0, to=1000, textvariable=count_var, width=5)
            spinbox.pack(side='right')

            self.item_counts[item] = count_var

        submit_button = ttk.Button(self, text="Submit", command=self.on_submit)
        submit_button.pack(pady=10)

    def on_submit(self):
        for item, var in self.item_counts.items():
            self.item_counts[item] = var.get()
        self.destroy()

def get_initial_counts():
    root = tk.Tk()
    root.withdraw()
    dialog = ItemCountInitializer(root)
    root.wait_window(dialog)
    return dialog.item_counts

# Function to display final counts (unchanged)
def show_final_counts(item_count):
    result_window = tk.Toplevel()
    result_window.title("Final Item Counts")
    result_window.geometry("300x400")

    tree = ttk.Treeview(result_window, columns=('Item', 'Count'), show='headings')
    tree.heading('Item', text='Item')
    tree.heading('Count', text='Count')
    tree.pack(fill='both', expand=True)

    for item, count in item_count.items():
        tree.insert('', 'end', values=(item, count))

    close_button = ttk.Button(result_window, text="Close", command=result_window.destroy)
    close_button.pack(pady=10)

    result_window.mainloop()

# Get initial item counts from user
item_count = get_initial_counts()

# Initialize your video capture, model, and other necessary parts here
# ...

mid_line = height*9// 15
line_color = (255, 0, 0)  # Red color in BGR
line_thickness = 2

# Dictionary to track objects
trackers = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    cv2.line(frame, (0, mid_line), (width, mid_line), line_color, line_thickness)

    # Apply model to each frame
    predictions = model.predict(frame)
    for prediction in predictions:
        x, y, w, h = int(prediction['x']), int(prediction['y']), int(prediction['width']), int(prediction['height'])
        center_y = y + h
        label = prediction['class']
        
        # Initialize count for new items
        if label not in item_count:
            item_count[label] = 0

        id_found = None

        # Find the closest existing tracker (simple overlap or centroid distance)
        for obj_id, (obj_label, obj_bbox, obj_last_y, obj_crossed) in trackers.items():
            if label == obj_label and not obj_crossed and np.abs(obj_last_y - center_y) < 300:  # 300 pixels threshold
                id_found = obj_id
                break

        if id_found is None:
            # New object detected
            new_id = max(trackers.keys(), default=-1) + 1
            trackers[new_id] = (label, (x, y, w, h), center_y, False)
        else:
            # Update existing tracker
            _, _, obj_last_y, obj_crossed = trackers[id_found]
            if center_y < mid_line and obj_last_y >= mid_line and not obj_crossed:
                item_count[label] -= 1
                trackers[id_found] = (label, (x, y, w, h), center_y, True)
            elif center_y > mid_line and obj_last_y <= mid_line and not obj_crossed:
                item_count[label] += 1
                trackers[id_found] = (label, (x, y, w, h), center_y, True)
            else:
                trackers[id_found] = (label, (x, y, w, h), center_y, obj_crossed)

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} {prediction['confidence']:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display item counts in the bottom-left corner of the frame
    text_position = (10, height - 10)
    for key, value in item_count.items():
        display_text = f"{key}: {value}"
        cv2.putText(frame, display_text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        text_position = (text_position[0], text_position[1] - 30)  # Adjust this value based on your font size

    # Write the frame into the file
    out.write(frame)

    # Clean up trackers that are no longer in view
    trackers = {obj_id: v for obj_id, v in trackers.items() if v[2] > 0 or v[2] < height}

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

# Show final counts in a GUI window
show_final_counts(item_count)

# 3.3 If the input video is not in 640*640 (resizing might take longer time)

In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk
from roboflow import Roboflow

# Predefined list of items (you can modify this list or read from a file)
KNOWN_ITEMS = ['Bottle', 'R_Lunch']

# Target size for video frames
TARGET_SIZE = (640, 640)

class ItemCountInitializer(tk.Toplevel):
    def __init__(self, parent):
        super().__init__(parent)
        self.title("Initialize Item Counts")
        self.geometry("300x400")
        self.item_counts = {}

        self.create_widgets()

    def create_widgets(self):
        for item in KNOWN_ITEMS:
            frame = ttk.Frame(self)
            frame.pack(pady=5, padx=10, fill='x')

            label = ttk.Label(frame, text=item)
            label.pack(side='left')

            count_var = tk.IntVar(value=0)
            spinbox = ttk.Spinbox(frame, from_=0, to=1000, textvariable=count_var, width=5)
            spinbox.pack(side='right')

            self.item_counts[item] = count_var

        submit_button = ttk.Button(self, text="Submit", command=self.on_submit)
        submit_button.pack(pady=10)

    def on_submit(self):
        for item, var in self.item_counts.items():
            self.item_counts[item] = var.get()
        self.destroy()

def get_initial_counts():
    root = tk.Tk()
    root.withdraw()
    dialog = ItemCountInitializer(root)
    root.wait_window(dialog)
    return dialog.item_counts

def show_final_counts(item_count):
    result_window = tk.Toplevel()
    result_window.title("Final Item Counts")
    result_window.geometry("300x400")

    tree = ttk.Treeview(result_window, columns=('Item', 'Count'), show='headings')
    tree.heading('Item', text='Item')
    tree.heading('Count', text='Count')
    tree.pack(fill='both', expand=True)

    for item, count in item_count.items():
        tree.insert('', 'end', values=(item, count))

    close_button = ttk.Button(result_window, text="Close", command=result_window.destroy)
    close_button.pack(pady=10)

    result_window.mainloop()

# Get initial item counts from user
item_count = get_initial_counts()

# Initialize your video capture
cap = cv2.VideoCapture('path_to_your_video.mp4')  # Replace with your video path

# Get original video properties
orig_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
orig_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Calculate scaling factors
scale_x = TARGET_SIZE[0] / orig_width
scale_y = TARGET_SIZE[1] / orig_height

# Initialize video writer with resized dimensions
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, fps, TARGET_SIZE)

# Initialize your model here
# model = ...

# Calculate the position of the middle line for the resized frame
mid_line = TARGET_SIZE[1] * 9 // 15
line_color = (255, 0, 0)  # Red color in BGR
line_thickness = 2

# Dictionary to track objects
trackers = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame to target size
    frame = cv2.resize(frame, TARGET_SIZE)

    cv2.line(frame, (0, mid_line), (TARGET_SIZE[0], mid_line), line_color, line_thickness)

    # Apply model to each frame
    predictions = model.predict(frame)
    for prediction in predictions:
        x, y, w, h = int(prediction['x']), int(prediction['y']), int(prediction['width']), int(prediction['height'])
        center_y = y + h // 2
        label = prediction['class']
        
        # Initialize count for new items
        if label not in item_count:
            item_count[label] = 0

        id_found = None

        # Find the closest existing tracker (simple overlap or centroid distance)
        for obj_id, (obj_label, obj_bbox, obj_last_y, obj_crossed) in trackers.items():
            if label == obj_label and not obj_crossed and np.abs(obj_last_y - center_y) < 300:  # 300 pixels threshold
                id_found = obj_id
                break

        if id_found is None:
            # New object detected
            new_id = max(trackers.keys(), default=-1) + 1
            trackers[new_id] = (label, (x, y, w, h), center_y, False)
        else:
            # Update existing tracker
            _, _, obj_last_y, obj_crossed = trackers[id_found]
            if center_y < mid_line and obj_last_y >= mid_line and not obj_crossed:
                item_count[label] += 1
                trackers[id_found] = (label, (x, y, w, h), center_y, True)
            elif center_y > mid_line and obj_last_y <= mid_line and not obj_crossed:
                item_count[label] -= 1
                trackers[id_found] = (label, (x, y, w, h), center_y, True)
            else:
                trackers[id_found] = (label, (x, y, w, h), center_y, obj_crossed)

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} {prediction['confidence']:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display item counts in the bottom-left corner of the frame
    text_position = (10, TARGET_SIZE[1] - 10)
    for key, value in item_count.items():
        display_text = f"{key}: {value}"
        cv2.putText(frame, display_text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        text_position = (text_position[0], text_position[1] - 30)  # Adjust this value based on your font size

    # Write the frame into the file
    out.write(frame)

    # Clean up trackers that are no longer in view
    trackers = {obj_id: v for obj_id, v in trackers.items() if v[2] > 0 or v[2] < TARGET_SIZE[1]}

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

# Show final counts in a GUI window
show_final_counts(item_count)